# Neural Artistic Style Transfer - Real Time Video

In [1]:
import torch
# If there are GPUs, choose the first one for computing. Otherwise use CPU.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)  
# If 'cuda:0' is printed, it means GPU is available.

cpu


## Load pre-trained Style CNN Model

In [8]:
from network.image_transformer_net import TransformerNet

model = TransformerNet()
model.load_state_dict(torch.load("models/starry_night.ckpt", map_location=device))

<All keys matched successfully>

## Video Neural Artistic Transfer

In [9]:
import torchvision.transforms as transforms
from torch.autograd import Variable

from PIL import Image
import imageio

imsize = 256

loader = transforms.Compose([
             transforms.Resize((imsize, imsize)),
             transforms.ToTensor()
         ])

unloader = transforms.ToPILImage()

In [10]:
import numpy as np
import cv2

# Convert numpy to tensor
def preprocess_image(image):
    image = loader(unloader(image[:, :, ::-1].copy()))
    image = image.unsqueeze(0)
    return image

In [11]:
cap = cv2.VideoCapture(0)
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

while(True):
    # Read image from camera
    ret, frame = cap.read()

    # Preprocess frame
    frame = cv2.resize(frame, (imsize, imsize))
    image = preprocess_image(frame).type(dtype)
    
    # Use trained model to produce neural artistic image
    pastiche = model(image)
    pastiche.data.clamp_(0, 1)
    image = pastiche.data.clone().cpu()
    image = image.view(3, imsize, imsize)
    image = unloader(image)

    frame = np.array(image)[:, :, ::-1].copy() # Convert RGB to BGR
    frame = cv2.resize(frame, (550, 500))
    
    # Show the image
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()